In [1]:
import numpy as np
import requests
import time

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

In [2]:
# Load the dataset and labels
x=np.load('NBx.npy')
y=np.load('NBy.npy')

In [3]:
conf = SparkConf()
sc = SparkContext(conf = conf)
spark = SQLContext(sc)

In [4]:
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=2) #The number of clusters is set to the number of classes in the dataset
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
    r=metrics.homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    R.append(r)

In [5]:
#Arrange feature accroding to thier ranks
Rnk=np.argsort(np.array(R))

In [6]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [7]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
et=0
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    lpa=np.zeros((0,2))
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        model = GradientBoostedTrees.trainClassifier(TrD,categoricalFeaturesInfo={})
        predictions = model.predict(TsD.map(lambda x: x.features))
        st = time.time()
        labelsAndPredictions = TsD.map(lambda lp: lp.label).zip(predictions)
        lpa=np.vstack((lpa,labelsAndPredictions.toDF().toPandas()))
        et+=time.time()-st
        acc = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]).count() / float(TsD.count())
        pp=pp+acc
    pp=pp/kfolds.n_splits
    np.savetxt('F%d.csv'%j,lpa,delimiter=',')
    smr.append([j, pp, et*1000000/x.shape[0]]) #Calculate the time required to predict a label per each object in uS.

In [8]:
smr

[[78.0, 0.9981, 1.2401],
 [77.0, 0.9985, 1.318],
 [76.0, 0.9983, 1.2225],
 [75.0, 0.9981, 1.2223],
 [74.0, 0.9989, 1.3394],
 [73.0, 0.9981, 1.3999],
 [72.0, 0.9986, 1.1684],
 [71.0, 0.9981, 1.4243],
 [70.0, 0.9988, 1.2597],
 [69.0, 0.9986, 1.1591],
 [68.0, 0.9945, 1.231],
 [67.0, 0.9984, 1.1837],
 [66.0, 0.9987, 1.1062],
 [65.0, 0.9985, 1.0832],
 [64.0, 0.9983, 1.0804],
 [63.0, 0.9987, 1.1406],
 [62.0, 0.9983, 1.0603],
 [61.0, 0.9985, 1.0587],
 [60.0, 0.9984, 0.9506],
 [59.0, 0.9984, 1.0178],
 [58.0, 0.9984, 0.9444],
 [57.0, 0.9989, 0.9628],
 [56.0, 0.9983, 0.9401],
 [55.0, 0.999, 1.0502],
 [54.0, 0.9984, 0.9177],
 [53.0, 0.9986, 0.8377],
 [52.0, 0.9985, 0.9818],
 [51.0, 0.999, 0.9122],
 [50.0, 0.9988, 0.8652],
 [49.0, 0.9982, 0.8509],
 [48.0, 0.9985, 0.858],
 [47.0, 0.9987, 0.8618],
 [46.0, 0.9986, 0.7688],
 [45.0, 0.9988, 0.809],
 [44.0, 0.9983, 0.783],
 [43.0, 0.9986, 0.7987],
 [42.0, 0.9653, 0.7841],
 [41.0, 0.9991, 0.8071],
 [40.0, 0.9654, 0.7811],
 [39.0, 0.9659, 0.8029],
 [38.0,